In [1]:
# let's dataset
corpus = [
          "We always come to Paris",
          "The professor is from Australia",
          "I live in Stanford",
          "He comes from Taiwan",
          "The capital of Turkey is Ankara"
         ]

Preprocessing the dataset for bare-minimum

In [2]:
# lowercasing the code
def preprocess_sentence(sentence) -> str:
    return sentence.lower().split() 

# creating the training set
train_sentences = [preprocess_sentence(text) for text in corpus]
train_sentences

[['we', 'always', 'come', 'to', 'paris'],
 ['the', 'professor', 'is', 'from', 'australia'],
 ['i', 'live', 'in', 'stanford'],
 ['he', 'comes', 'from', 'taiwan'],
 ['the', 'capital', 'of', 'turkey', 'is', 'ankara']]

In [3]:
# set locations to apperar in corpus
locations = set(["australia", "ankara", "paris", "stanford", "taiwan", "turkey"])

In [4]:
# training labels
train_labels = [[1 if word in locations else 0 for word in text] for text in train_sentences]
train_labels

[[0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1, 0, 1]]

Embeddings! Fuck

In [5]:
vocab = set(w for s in train_sentences for w in s)
vocab

{'always',
 'ankara',
 'australia',
 'capital',
 'come',
 'comes',
 'from',
 'he',
 'i',
 'in',
 'is',
 'live',
 'of',
 'paris',
 'professor',
 'stanford',
 'taiwan',
 'the',
 'to',
 'turkey',
 'we'}

In [6]:
# Add the unknown token to our vocabulary
vocab.add("<unk>")

In [7]:
vocab.add("<pad>")
def pad_window(sentence, window_size, pad_tokens="<pad>"):
    window = [pad_tokens] * window_size
    return window + sentence + window

window_size = 2
pad_window(train_sentences[0], window_size=window_size)

['<pad>', '<pad>', 'we', 'always', 'come', 'to', 'paris', '<pad>', '<pad>']

In [8]:
# we have to assign an idx to each vocab
ix_to_word = sorted(list(vocab))
word_to_ix = {word: ind for ind, word in enumerate(ix_to_word)}
word_to_ix
# enumerate lets us keep track something like fuck

{'<pad>': 0,
 '<unk>': 1,
 'always': 2,
 'ankara': 3,
 'australia': 4,
 'capital': 5,
 'come': 6,
 'comes': 7,
 'from': 8,
 'he': 9,
 'i': 10,
 'in': 11,
 'is': 12,
 'live': 13,
 'of': 14,
 'paris': 15,
 'professor': 16,
 'stanford': 17,
 'taiwan': 18,
 'the': 19,
 'to': 20,
 'turkey': 21,
 'we': 22}

In [9]:
# covert them to indiuces
def convert_token_to_indices(sentence, word_to_ix):
    indices = []
    for token in sentence:
        if token in word_to_ix:
            index = word_to_ix[token]
        else:
            index = word_to_ix["<unk>"]
        indices.append(index)
    return indices

In [10]:
example_sentence = ["we", "always", "come", "to", "kuwait"]
example_indices = convert_token_to_indices(example_sentence, word_to_ix)
restored_example = [ix_to_word[ind] for ind in example_indices]

In [11]:
print(f"Original sentence is: {example_sentence}")
print(f"Going from words to indices: {example_indices}")
print(f"Going from indices to words: {restored_example}")

Original sentence is: ['we', 'always', 'come', 'to', 'kuwait']
Going from words to indices: [22, 2, 6, 20, 1]
Going from indices to words: ['we', 'always', 'come', 'to', '<unk>']


In [12]:
example_padded_indices = [convert_token_to_indices(s,word_to_ix) for s in train_sentences]

In [13]:
example_padded_indices

[[22, 2, 6, 20, 15],
 [19, 16, 12, 8, 4],
 [10, 13, 11, 17],
 [9, 7, 8, 18],
 [19, 5, 14, 21, 12, 3]]

In [14]:
# creating the embedding table
embedding_dim = 5
import torch.nn as nn
embeds = nn.Embedding(len(vocab), embedding_dim)

Emperical Formula for determining Embedding dim

Embed_dim = (num_of_word_or_elements ** 0.25) # or NUM OF ELEMENTS ^(1/2) 

In [18]:
list(embeds.parameters())

[Parameter containing:
 tensor([[ 0.5686,  0.9715,  0.0451,  1.5931, -1.4669],
         [-0.5358, -1.2165,  0.3604, -0.4787, -1.4749],
         [ 2.5492, -0.2397, -0.1668,  0.5925,  0.6002],
         [-0.1383,  0.9898, -0.4856,  1.3780,  0.2166],
         [ 0.4921,  0.1460,  0.8977, -1.2567, -0.3184],
         [ 1.4324, -0.1228,  1.3989, -0.0445,  0.1734],
         [-1.0744,  1.0244, -0.3263,  1.8729, -0.4322],
         [ 0.2626, -0.7837,  0.8404, -0.6829,  1.2088],
         [ 0.2452,  1.4278, -0.0186, -0.1464, -0.6267],
         [ 2.1342,  1.3594,  0.3160, -0.0366, -1.5533],
         [ 0.8921,  1.8061,  0.4301, -1.1312,  0.6416],
         [-1.4366,  0.5967,  1.0330,  0.3859,  1.1610],
         [-1.2120,  0.9837, -0.0352, -0.5095,  0.7749],
         [ 0.2758,  0.2709, -0.2766,  1.6259,  0.0664],
         [ 0.6223,  1.3239,  0.9706, -0.4761, -0.6280],
         [ 0.5972, -2.5759, -0.3597,  0.6420, -0.2552],
         [ 1.2539, -0.3473,  2.4494, -0.0641, -1.0340],
         [ 0.6382, -0.414

In [20]:
# get the embeddings for the word Paris
index = word_to_ix["paris"]
import torch 
index_tensor = torch.tensor(index, dtype=torch.long)
paris_embed = embeds(index_tensor)
paris_embed

tensor([ 0.5972, -2.5759, -0.3597,  0.6420, -0.2552],
       grad_fn=<EmbeddingBackward0>)

In [21]:
# We can also get multiple embeddings at once
index_paris = word_to_ix["paris"]
index_ankara = word_to_ix["ankara"]
indices = [index_paris, index_ankara]
indices_tensor = torch.tensor(indices, dtype=torch.long)
embeddings = embeds(indices_tensor)
embeddings

tensor([[ 0.5972, -2.5759, -0.3597,  0.6420, -0.2552],
        [-0.1383,  0.9898, -0.4856,  1.3780,  0.2166]],
       grad_fn=<EmbeddingBackward0>)

In [22]:
def custom_collate_fn(batch, window_size, word_to_ix):
    x, y = zip(*batch)
    x = [pad_window(s, window_size=window_size) for s in x]
    x = [convert_token_to_indices(s, word_to_ix) for s in x]
    
    ## pading so that it has all sentences with same size
    pad_token_ix =  word_to_ix["<pad>"]
    x = [torch.LongTensor(x_i) for x_i in x]
    x_padded = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=pad_token_ix)
    
    # batch first means batch size will be first and then seq_len
    
    # pad y and record the length 
    lengths = [len(label) for label in y]
    lengths = torch.LongTensor(lengths)
    y = [torch.LongTensor(y_i) for y_i in y]
    y_padded = nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=0)
    
    return x_padded, y_padded, lengths
    

In [23]:
data = list(zip(train_sentences, train_labels))
batch_size = 5
shuffle = True 
window_size = 2
from functools import partial
collate_fn = partial(custom_collate_fn, window_size=window_size, word_to_ix=word_to_ix)

In [25]:
from torch.utils.data import DataLoader; loader = DataLoader(data, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)

In [26]:
counter = 0
for batched_x, batched_y, batched_lengths in loader:
    print(f'Iteration {counter}')
    print(f'Batched INPUT: ')
    print(batched_x)
    print("Batched Labels: ")
    print(batched_y)
    print("Batched lengths: ")
    print(batched_lengths)
    print("")
    counter += 1

Iteration 0
Batched INPUT: 
tensor([[ 0,  0,  9,  7,  8, 18,  0,  0,  0,  0],
        [ 0,  0, 22,  2,  6, 20, 15,  0,  0,  0],
        [ 0,  0, 19,  5, 14, 21, 12,  3,  0,  0],
        [ 0,  0, 19, 16, 12,  8,  4,  0,  0,  0],
        [ 0,  0, 10, 13, 11, 17,  0,  0,  0,  0]])
Batched Labels: 
tensor([[0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0]])
Batched lengths: 
tensor([4, 5, 6, 5, 4])



In [27]:
# printing the original tensor 
print("original tensor")
print(batched_x)
print("")

original tensor
tensor([[ 0,  0,  9,  7,  8, 18,  0,  0,  0,  0],
        [ 0,  0, 22,  2,  6, 20, 15,  0,  0,  0],
        [ 0,  0, 19,  5, 14, 21, 12,  3,  0,  0],
        [ 0,  0, 19, 16, 12,  8,  4,  0,  0,  0],
        [ 0,  0, 10, 13, 11, 17,  0,  0,  0,  0]])



In [28]:
# create the 2 *2 + 1 chunks 
chunk = batched_x.unfold(1, window_size * 2 + 1, 1)
print(f"Windows: ")
print(chunk)

Windows: 
tensor([[[ 0,  0,  9,  7,  8],
         [ 0,  9,  7,  8, 18],
         [ 9,  7,  8, 18,  0],
         [ 7,  8, 18,  0,  0],
         [ 8, 18,  0,  0,  0],
         [18,  0,  0,  0,  0]],

        [[ 0,  0, 22,  2,  6],
         [ 0, 22,  2,  6, 20],
         [22,  2,  6, 20, 15],
         [ 2,  6, 20, 15,  0],
         [ 6, 20, 15,  0,  0],
         [20, 15,  0,  0,  0]],

        [[ 0,  0, 19,  5, 14],
         [ 0, 19,  5, 14, 21],
         [19,  5, 14, 21, 12],
         [ 5, 14, 21, 12,  3],
         [14, 21, 12,  3,  0],
         [21, 12,  3,  0,  0]],

        [[ 0,  0, 19, 16, 12],
         [ 0, 19, 16, 12,  8],
         [19, 16, 12,  8,  4],
         [16, 12,  8,  4,  0],
         [12,  8,  4,  0,  0],
         [ 8,  4,  0,  0,  0]],

        [[ 0,  0, 10, 13, 11],
         [ 0, 10, 13, 11, 17],
         [10, 13, 11, 17,  0],
         [13, 11, 17,  0,  0],
         [11, 17,  0,  0,  0],
         [17,  0,  0,  0,  0]]])


In [31]:
class WordWindowClassifier(nn.Module):
    def __init__(self, hyperparameters, vocab_size, pad_ix=0):
        super().__init__()
        self.window_size = hyperparameters["window_size"]
        self.embed_dim = hyperparameters["embed_dim"]
        self.hidden_dim = hyperparameters["hidden_dim"]
        self.freeze_embeddings = hyperparameters["freeze_embedding"]
        
        self.embeds = nn.Embedding(vocab_size, self.embed_dim, padding_idx=pad_ix)
        if self.freeze_embeddings:
            self.embeds.weight.requires_grad = False

        full_window_size = 2 * self.window_size + 1
        self.hidden_layer = nn.Sequential(
            nn.Linear(full_window_size * self.embed_dim, self.hidden_dim),
            nn.Tanh()
        )

        self.output_layer = nn.Linear(self.hidden_dim, 1)
        self.probabilities = nn.Sigmoid()
        
    def forward(self, inputs):
        B, L = inputs.size()
        token_windows = inputs.unfold(1, 2 * self.window_size + 1, 1)
        _, adjusted_length, window_size = token_windows.size()
        assert token_windows.size() == (B, adjusted_length, 2 * self.window_size + 1)

        embedded_windows = self.embeds(token_windows)
        embedded_windows = embedded_windows.view(B, adjusted_length, -1)
        layer1 = self.hidden_layer(embedded_windows)
        output = self.output_layer(layer1)
        output = self.probabilities(output)
        output = output.view(B, -1)
        return output

In [33]:
data = list(zip(train_sentences, train_labels))
batch_size = 3
shuffle = True
window_size = 2
collate_fn = partial(custom_collate_fn, window_size=window_size, word_to_ix=word_to_ix)

# Instantiate a DataLoader
loader = DataLoader(data, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)

# Initialize a model
# It is useful to put all the model hyperparameters in a dictionary
model_hyperparameters = {
    "batch_size": 4,
    "window_size": 2,
    "embed_dim": 25,
    "hidden_dim": 25,
    "freeze_embedding": False,
}

vocab_size = len(word_to_ix)
model = WordWindowClassifier(model_hyperparameters, vocab_size)

# Define an optimizer
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [39]:
def loss_function(batch_outputs, batch_labels, batch_lengths):
    bceloss = nn.BCELoss()
    loss = bceloss(batch_outputs, batch_labels.float())
    # rescale loss
    # number of words in each training example
    loss = loss / batch_lengths.sum().float()
    return loss 
    

In [40]:
def train_epoch(loss_function, optimizer, model, loader):
    # keep track of total loss for batch
    total_loss = 0
    for batch_inputs, batch_labels, batched_lengths in loader:
        # clear gradients 
        optimizer.zero_grad()
        outputs = model.forward(batch_inputs)
        loss = loss_function(outputs, batch_labels, batched_lengths)
        # calculate gradients
        loss.backward()
        # update the parameters
        optimizer.step()
        total_loss += loss.item()
    return total_loss

def train(loss_function, optimizer, model, loader, num_epochs=10):
    for epoch in range(num_epochs):
        epoch_loss = train_epoch(loss_function, optimizer, model, loader)
        if epoch % 100 == 0: print(epoch_loss)

In [43]:
num_epochs = 100000
train(loss_function, optimizer, model, loader, num_epochs=num_epochs)

0.05682901665568352
0.04459817335009575
0.04280679300427437
0.036663344129920006
0.03145122807472944
0.03407581336796284
0.026276499032974243
0.02902739029377699
0.027926050126552582
0.02056761644780636
0.023901347536593676
0.01599164167419076
0.01676288153976202
0.018284517340362072
0.014100173022598028
0.015149241778999567
0.014220210257917643
0.012395231053233147
0.013274933211505413
0.012863545212894678
0.011986434925347567
0.010154706425964832
0.01146731711924076
0.009751930832862854
0.009300614707171917
0.009944861056283116
0.007339452393352985
0.007013353751972318
0.005826340522617102
0.007340805605053902
0.009023855673149228
0.006835629232227802
0.009304615436121821
0.009014354320243001
0.006179980002343655
0.005101962597109377
0.005879396805539727
0.005633634282276034
0.004003340960480273
0.005936676636338234
0.006601099856197834
0.004188288934528828
0.004063788102939725
0.006868573487736285
0.004578335676342249
0.005040105897933245
0.0036285044625401497
0.0030596618307754397


KeyboardInterrupt: 